In [2]:
import pandas as pd

The 18 identifiers that make health information PHI are:

 - Names
 - Dates, except year
 - Telephone numbers
 - Geographic data
 - FAX numbers
 - Social Security numbers
 - Email addresses
 - Medical record numbers
 - Account numbers
 - Health plan beneficiary numbers
 - Certificate/license numbers
 - Vehicle identifiers and serial numbers including license plates
 - Web URLs
 - Device identifiers and serial numbers
 - Internet protocol addresses
 - Full face photos and comparable images
 - Biometric identifiers (i.e. retinal scan, fingerprints)
 - Any unique identifying number or code

In [5]:
deid_dict = {
    "de-identification_root_concept": ["name", "contact_details", "healthcare_identifiers", "date"],
    "name": ["fore_name", "surname"],
    "contact_details": ["address", "telephone_number", "email", "identification"],
    "address": ["address_line", "postcode"],
    "identification": ["passport_number", "driving_licence_number"],
    "healthcare_identifier": ["nhs_number", "hospital_number", "emergency_department_number"],
    "date": ["date_of_birth"],
    }


In [7]:
print(deid_dict.keys())


dict_keys(['de-identification_root_concept', 'name', 'contact_details', 'address', 'identification', 'healthcare_identifier', 'date'])


#### Meta_annotations will hold the contextual information of the concept.
"Subject": ["patient", "family", "clinician"]


In [ ]:
# TODO: visualise the terminology hierarchy
# TODO: Create a JSON file with each concept and concept unique identifier
# TODO: enrich the terminology with synonyms
